In [ ]:
import os
from urllib import request

In [ ]:
if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
    file_path = "the-verdict.txt"
    request.urlretrieve(url, file_path)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

raw_text[:20]

'I HAD always thought'

### Get Tokens from Dataset

In [6]:
import re
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
print(result[:20])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was']


In [12]:
all_words = sorted(set(result))
vocab_size = len(all_words)

vocab = {word:integer for integer, word in enumerate(all_words)}
vocab["HAD"]

44

### Creating Tokenizer

In [ ]:
class Tokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for i, s in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # adding special context token, 
        # handles tokens that are not existed in vocabulary
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = "".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = Tokenizer(vocab=vocab)

### Byte Pair Encoding

In [ ]:
!pip install tiktoken

In [14]:
import tiktoken

In [ ]:
# to handle more complicated texts, sometimes meaningless
adv_tokenizer = tiktoken.get_encoding("gpt2")
adv_tokenizer.encode("Hello World~!")

[15496, 2159, 93, 0]

### Data Sampling with Sliding Window

In [17]:
encoded_text = adv_tokenizer.encode(raw_text)
encoded_text[:10]

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]

In [ ]:
context_size = 1
x = encoded_text[:context_size]
y = encoded_text[1:context_size+1]

In [ ]:
!pip install torch

In [19]:
import torch
torch.__version__

'2.8.0+cpu'

In [44]:
from torch.utils.data import Dataset, DataLoader

class LLMDataset(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [45]:
def create_dataloader(text, batch_size=4, max_length=256, stride=128,
                      shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = LLMDataset(text, tokenizer, max_length, stride)

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                            shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [50]:
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("inputs: ", inputs)
print("targets: ", targets)

inputs:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
targets:  tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Creating Token Embeddings

In [54]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(adv_tokenizer.n_vocab, output_dim)

embedding_layer.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035],
        [-0.5880,  0.3486,  0.6603],
        [-0.2196, -0.3792,  0.7671],
        ...,
        [-0.5931,  1.0895, -0.6854],
        [ 0.7447,  0.5803, -0.4246],
        [-0.3130,  0.7558, -1.2656]], requires_grad=True)

### Encode Word Postion

In [58]:
vocab_size = adv_tokenizer.n_vocab
output_dim = 256

embedding_layer = torch.nn.Embedding(adv_tokenizer.n_vocab, output_dim)

token_embeddings = embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [59]:
context_length = 4 # equal to max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [60]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])